# 1. 乱数の発生方法

モンテカルロ法で利用する乱数の発生方法とその性質について。

## そもそも、乱数とは

- 乱数(random value)とは、サイコロの出目のように規則性がなく予測不能な数値のこと。何度も生成した時に、すでに分かっている値の列から次に現れる値を予測できないような数値の列を乱数列と呼び、その中の個々の値を乱数という。（[IT用語辞典](https://e-words.jp/w/%E4%B9%B1%E6%95%B0.html)）

## 疑似乱数

- 一見乱数のように規則性のない数の並びのように見えるが、一定の計算手順によって確定的に与えられる数値の列。また、その中の個々の数値。
    - 擬似乱数は初期値（シード値）と計算手順が同じなら同じ数列が得られるため、数値シミュレーションなどではまったく同じ環境・状態の再現が可能となる。
- 疑似乱数が満たすべき条件は以下の4つ：
    1. 周期が長いこと
    2. 統計的検定に耐えること（周期が長くても、規則性があっては意味がない）
    3. 再現性があること（同じ乱数の種（初期値）を使えば、同じ乱数が出力できる）
    4. 乱数発生のスピードが早いこと

## Julia の乱数発生機構

- Julia には、乱数発生器として `Random.rand()` 関数が用意されている。
    - [Random Numbers](https://docs.julialang.org/en/v1/stdlib/Random/)
- 関数 `rand()` の使い方：
    - 第1引数：乱数の範囲のコレクション
    - 第2引数：出力する乱数の個数（行列でも良い）
- 以下、出力例：

In [1]:
rand()

0.5455646495582072

In [2]:
rand(5)

5-element Vector{Float64}:
 0.39235536810714744
 0.446152770454493
 0.34271016675278876
 0.4412867129667668
 0.32237920384584573

In [3]:
rand(1:6, 10)

10-element Vector{Int64}:
 4
 1
 4
 6
 4
 6
 5
 1
 5
 3

In [4]:
rand(Int, 3)

3-element Vector{Int64}:
 -6530542584229124820
  -706045200000852418
 -7496776384956152571

In [5]:
rand(Float64, (2,3))

2×3 Matrix{Float64}:
 0.666195  0.071927  0.720947
 0.813906  0.334357  0.661256

- `Random.seed!()` を使って、乱数の種を指定することが可能

In [6]:
using Random

Random.seed!(10)
rand()

0.11258244478647295

In [7]:
using Random

Random.seed!(10)
rand()

0.11258244478647295

## 頻度検定

- 上記の関数で発生させた乱数が一様に分布しているかを調べる。
- 処理の概要は以下の通り：
    - `Random.rand(1:10)` で 1 ~ 10 の乱数を 10000 個発生させる
    - 1 ごとの刻み幅：0 ~ 1, 1 ~ 2, 2 ~ 3,…にそれぞれ何個の乱数が入るかをカウントする

### 理論分布との比較

- 今回の理論分布 $f_0$ は $10000 / 10 = 1000$ となるはず。
- 一般に統計学では母集団をサンプリングして得られた分布と、既知の分布との間に差があるかどうかを判定することを、**適合度の検定**（以下では単に**検定**）と言う。
- 今回の一様乱数発生の例では、サンプリング・データは $f(i)$ 、既知の分布は $f_0 = 1000$ に相当する。

### 検定

- 検定は次のように行う：
    - $f_0$: 期待値
    - $f(k)$: 実現値

$$
    \sum_{k=1}^{n}\frac{(f(k) - f_0)^2}{f_0}
$$

- 今回の例で書けば以下：
$$
    \frac{(f(1) - f_0)^2}{f_0} + \frac{(f(2) - f_0)^2}{f_0} + ... + \frac{(f(10) - f_0)^2}{f_0}
$$

- これを通常 $\chi^2$（カイ二乗）といい、サンプリングデータと理論度数との食い違いの程度を表す。
    - サンプリングデータと理論度数の食い違いが全くなければ、$\chi^2 = 0$ となり、食い違いが大きいほど $\chi^2$ は大きくなる。
- 同じ母集団からのサンプリングのやり方は、何通りもあるので、　$\chi^2 = 0$ は **$\chi^2$ 分布** と呼ばれる分布に従う。
- $\chi^2$ 分布は自由度と呼ばれるパラメータを持つ
    - 今回の場合は 9 （刻みの数 - 1）である。
    - [分布の形](https://images.app.goo.gl/k3k6CTGkPkEDzTSC9)
    - [分布表](http://www3.u-toyama.ac.jp/kkarato/2016/statistics/handout/chisqdist.pdf)
        - サンプリング・データから得られた $\chi^2$ が　$\chi_{\alpha}^2$ より小さければ、”危険率 $\alpha$ で理論度数とサンプリング・データが等しい”と言えることになる

In [8]:
using Printf

num = 10000         # 発生させる乱数の数
f = zeros(Int, 10)  # 刻み i に落ちる乱数の数
xs = 0.0            # カイ二乗の値
f0 = num / 10.0     # 理論度数

for j = 1:num
    i = rand(1:10)
    f[i] += 1
end

@printf "%2s %5s\n" "i" "f[i]"

# 頻度の検定
for i = 1:10
    @printf "%2d %5d\n" i  f[i]
    xs = xs + (f[i] - f0)^2 / f0
end

println("xs = $xs")

 i  f[i]
 1   995
 2  1014
 3   936
 4   976
 5  1020
 6  1011
 7  1001
 8  1036
 9  1010
10  1001
xs = 6.812


- プログラムの `xs` は $\chi^2$ を計算している。
- 分布表より、自由度 9 における危険率 5% の $\chi_{\alpha}^2$ の値は、 16.919 なので、`Random.rand(1:10)` メソッドは頻度テストは合格したことになる。

## ポーカー検定(poker test)

- ポーカー検定は、無規則性の検定の手法
    - 繰り返しが規則的で、乱数とは言えない状態ではないかを検証する
- 整数の乱数の列を連続して 5 つずつまとめ、5 桁の数字とみて組を作り、各組を 7 つの型に分類し、各種類の 5 個組の数についてカイ2乗検定を行う
    1. abcde(すべて異種)
    2. aabcd(ペアが1組ある)
    3. aabbc(ペアが2組ある)
    4. aaabc(3個同種)
    5. aaabb(3個同種とペアが1組)
    6. aaaab(4個同種)
    7. aaaaa(5個同種)
- これらの起こる確率は理論的に求められ、[表](https://www.ishikawa-lab.com/montecarlo/jpg/table2-3-1.jpg)のとおりとなる。

### ポーカー検定のプログラム

- 大きな流れは以下の通り：
    - `Random.rand(0:9)` で 5 個の整数の並びを作成：`n[1], n[2], n[3], n[4], n[5]` とする
- 次に、下の図のとおり比較を行い、同じ数字であれば `count` を 1 増加させる
    - 全部同じ数字であれば `count = 10` となり、全部異なれば `count = 0` となる
    ![](https://www.ishikawa-lab.com/montecarlo/jpg/fig2-4-11.jpg)
- 上記で挙げた 7 つの型に対して、 `count` は[表](https://www.ishikawa-lab.com/montecarlo/jpg/table2-4-1.jpg)の値を取るので、それを「型の番号」である `i` に変換する。
- 頻度 `f[i]` をカウントし、$\chi^2$ を以下により求め, 自由度 6 の $\chi^2$ 分布表を用いて検定する：

$$
    \chi^2 = \sum_{i=1}^{7}\frac{(f(i) - f_0(i))^2}{f_0(i)}
$$

In [9]:
using Printf

num = 1000000       # 発生させる乱数の数
f = zeros(Int, 7)   # iごとの度数
f0 = zeros(Int, 7)  # 理論度数
n = zeros(Int, 5)   # 5個の数字の並び
xs = 0.0            # カイ二乗の値

f0[1] = num * 0.3024
f0[2] = num * 0.5040
f0[3] = num * 0.1080
f0[4] = num * 0.0720
f0[5] = num * 0.0090
f0[6] = num * 0.0045
f0[7] = num * 0.0001

for k = 1:num
    for j = 1:5
        n[j] = rand(0:9)
    end
    
    count = 0
    
    for j1 = 1:4
        for j2 = (j1+1):5
            if n[j1] == n[j2]
                count += 1
            end
        end
    end
    
    if count == 10 
        i = 7
    elseif count == 6
        i = 6
    else
        i = count + 1
    end
    
    f[i] += 1
    
end

@printf "%2s %10s %10s\n" "i" "f[i]" "f0[i]"

for i = 1:7
    @printf "%2d %10d %10d\n" i f[i] f0[i]
    diff = f[i] - f0[i]
    xs = xs + diff^2 / f0[i]
end

println("xs = $xs")

 i       f[i]      f0[i]
 1     302399     302400
 2     504318     504000
 3     107806     108000
 4      71890      72000
 5       9024       9000
 6       4472       4500
 7         91        100
xs = 1.7654054232804235


- `Random.rand()` はポーカーテストにおいても合格する

## 演習

1. 1 から 6 までの整数が等確率で出現する、さいころのシミュレーションを行ないなさい。さいころをふるたびに、でたらめな数がでるようにしなさい。
2. 1 桁の区間 0 から 9 までの乱数があったとき、同じ数が現われるまでの間隔を、ギャップといいます。その長さが $r$ である確率 $p$ は、
$$
    p = \left( 1 - \frac{1}{10}\right)^{r} \cdot\frac{1}{10}
$$
で表されます。（$r = 0$ は、続いて同じ数字が現われた場合）。このことを用いて、 `Random.rand()` メソッドの乱数のギャップ検定を行ないなさい。

### 演習1 解答

In [10]:
rand(1:6, 10)

10-element Vector{Int64}:
 6
 6
 4
 3
 4
 4
 3
 2
 1
 3

### 演習2 解答

In [11]:
using Printf

max = 9               # 区間の最大の数
num = 10000           # 乱数の数
an = rand(0:max, num) # 0~max の乱数の列
f = zeros(Int, num)   # ギャップ r になった頻度
xs = 0.0              # カイ二乗数

# ギャップの長さが r である確率
function p(r::Int)
    return (1 - (1/(max+1)))^r * (1/(max+1))
end

r = 0 # ギャップ

for j = 0:max
    for i = 1:num
        if (an[i] == j)
            f[r+1] += 1
            r = 0
        else
            r += 1
        end
    end
end

@printf "%4s %10s %10s\n" "r" "f[r]" "f0[r]"

for r = 0:19
    f0 = p(r) * num # 理論値の計算
    @printf "%4d %10d %10d\n" r f[r+1] f0
    xs = xs + (f[r+1] - f0)^2 / f0
end

println("xs = $xs")

   r       f[r]      f0[r]
   0        993       1000
   1        913        900
   2        797        810
   3        765        729
   4        660        656
   5        501        590
   6        549        531
   7        481        478
   8        439        430
   9        413        387
  10        368        349
  11        313        314
  12        254        282
  13        274        254
  14        231        229
  15        204        206
  16        199        185
  17        167        167
  18        144        150
  19        136        135
xs = 25.046889639353665
